# Testing the logsumexp function
Try with mock data to see how it works, try to compute the exp(logp), define a mock data for the logp it should have small values. it doesnt even need to be a real prob density just a array with small numbers. 

for score estimator we need expectation of ( p_joint(we have logp so we do exp(logp)) * grad_z p(G|Z)) we can seperate the expectation terms (try!)

after having the score estimator grad for p_joint,
calculate numerator (score estimator grad_z) divided by expectation_p_joint ( we again have logp) 

doing num / den is hard again so do lognum - log den and then exponentiate

start first how to calculate stably the logp to p 

then by using the function for logp to p stable logsumexp trick 

try to use for the pjoint in denominator too 

and also for the finally for the ratio num / den

In [26]:
import torch


## negative values for logp 
# variables are, values of the entries and the shape of the tensor

val = -1000
size = 500

logp = torch.tensor([val] * size)
print(f'logp: {logp}')


# directly exponentiate the logp 
exp_logp = torch.exp(logp)
print(f'exp_logp: {exp_logp}')

# print the sum of the exp_logp
print(f'sum of exp_logp: {exp_logp.sum()}')
# print the log of the sum of the exp_logp
print(f'log of sum of exp_logp: {exp_logp.sum().log()}')

# which is equal to 
log_sum_exp = torch.logsumexp(logp,dim=0)
print(f'logsumexp of logp: {log_sum_exp}')






logp: tensor([-1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000,
        -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000,
        -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000,
        -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000,
        -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000,
        -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000,
        -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000,
        -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000,
        -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000,
        -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000,
        -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000,
        -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000,
        -1000, -1000, -1000, -1000, -1000, -1000, -1000, -

In [25]:
## but here where is the part if the value is negative what is log of a negative v

# log of a negative number is undefined
dummy_tensor = torch.tensor([-4])
print(f'log of a negative number: {torch.log(dummy_tensor)}')




# How about if we just reverse the sign and then do the logsumexp and then reverse the sign again?

sign_dummy_tensor = torch.sign(dummy_tensor)
print(f'sign of a negative number: {sign_dummy_tensor}')

neg_dummy_tensor = -1 * dummy_tensor
print(f'negative of a negative number: {neg_dummy_tensor}')

# final neg value 
log_dummy_tensor = torch.log(neg_dummy_tensor)
print(f'log of a negative number: {log_dummy_tensor}')

log_neg_dummy_tensor_reverse = sign_dummy_tensor * log_dummy_tensor
print(f'log of a negative number with reverse sign: {log_neg_dummy_tensor_reverse}')

# so some sign trick is needed to make it work 
# continue with this idea for the logsumexp of the numerator / denominator 
# (especially since numerator is a gradient with negative values)
 




log of a negative number: tensor([nan])
sign of a negative number: tensor([-1])
negative of a negative number: tensor([4])
log of a negative number: tensor([1.3863])
log of a negative number with reverse sign: tensor([-1.3863])


# Let's try with the expectation of a probability

In [39]:
val = -1000
size = 500

logp = torch.tensor([val + torch.randn(1) for _ in range(size)])
print(f'logp: {logp}')

log_p_max = torch.max(logp)
print(f'log_p_max: {log_p_max}')

log_p_shift = logp - log_p_max
print(f'log_p_shift: {log_p_shift}')
p_shift = torch.exp(log_p_shift)
print(f'p_shift: {p_shift}')

p = torch.exp(log_p_shift) * torch.exp(log_p_max)
print(f'p total: {p}')


logp: tensor([-1001.0804,  -998.3104,  -999.5331, -1000.1557, -1000.3253, -1000.9654,
        -1000.6677,  -999.9363, -1000.0333,  -999.7894,  -999.3666, -1000.4529,
        -1000.6130,  -999.6716,  -999.4961, -1000.4133,  -998.3268,  -999.3607,
         -999.3165,  -999.3884, -1000.0940,  -998.6558, -1000.4788,  -998.0553,
         -999.9222,  -999.3783,  -998.3589,  -999.2471, -1000.4186, -1000.1543,
        -1001.0948,  -999.6008,  -999.2224,  -999.6808, -1000.1816, -1001.9860,
         -999.8506,  -999.4153,  -999.3735, -1001.0857, -1000.4762,  -998.7845,
         -999.3394,  -999.2870,  -999.3116,  -998.5846, -1002.2172,  -999.0669,
         -999.6246,  -998.6852,  -998.3542, -1001.7344,  -999.7136, -1001.9893,
         -999.4598, -1001.2610, -1002.1068, -1001.2159, -1001.2245, -1001.0909,
        -1000.7559, -1001.5068, -1000.5159, -1000.4258, -1000.3645, -1000.4149,
         -999.0513,  -999.8271,  -999.7773,  -997.3482, -1000.3988,  -999.9426,
         -999.2041, -1000.4899, -1

In [40]:
## using the formula to calculate the score estimator 

# smth like expectation under (p G|Z) of = p(G|Z) * grad_z p(G|Z)

In [41]:
import torch
import torch.nn.functional as F

def dummy_score_estimator_stable_vs_unstable():
    """
    Demonstrates numerically stable vs unstable score estimator computation
    using dummy values to show the difference.
    """
    print("=" * 60)
    print("SCORE ESTIMATOR: STABLE VS UNSTABLE COMPUTATION")
    print("=" * 60)
    
    # Set random seed for reproducibility
    torch.manual_seed(42)
    
    # Dummy parameters
    n_samples = 5
    d = 3  # dimension of z
    z_dim = (d, 2, 2)  # shape of z tensor
    
    # Generate dummy log probabilities (very small values to demonstrate instability)
    log_p_values = torch.tensor([-1000.5, -1001.2, -999.8, -1002.1, -1000.9])
    print(f"Original log probabilities: {log_p_values}")
    
    # Generate dummy gradient values
    grad_values = torch.randn(n_samples, *z_dim) * 0.1
    print(f"Gradient values shape: {grad_values.shape}")
    print(f"Sample gradient values:\n{grad_values[0]}")
    
    print("\n" + "-" * 50)
    print("METHOD 1: NUMERICALLY UNSTABLE")
    print("-" * 50)
    
    # Unstable method: direct exponentiation
    try:
        p_unstable = torch.exp(log_p_values)
        print(f"Direct exp(log_p): {p_unstable}")
        print(f"Sum of exp(log_p): {p_unstable.sum()}")
        
        # Normalize to get weights
        weights_unstable = p_unstable / p_unstable.sum()
        print(f"Weights (unstable): {weights_unstable}")
        print(f"Sum of weights: {weights_unstable.sum()}")
        
        # Compute weighted gradient
        weighted_grad_unstable = torch.zeros_like(grad_values[0])
        for i in range(n_samples):
            weighted_grad_unstable += weights_unstable[i] * grad_values[i]
        
        print(f"Weighted gradient (unstable):\n{weighted_grad_unstable}")
        
    except Exception as e:
        print(f"Unstable method failed: {e}")
    
    print("\n" + "-" * 50)
    print("METHOD 2: NUMERICALLY STABLE")
    print("-" * 50)
    
    # Stable method: logsumexp trick
    log_p_max = torch.max(log_p_values)
    print(f"Max log probability: {log_p_max}")
    
    log_p_shifted = log_p_values - log_p_max
    print(f"Shifted log probabilities: {log_p_shifted}")
    
    p_shifted = torch.exp(log_p_shifted)
    print(f"Exp of shifted log_p: {p_shifted}")
    
    # Normalize to get weights
    weights_stable = p_shifted / p_shifted.sum()
    print(f"Weights (stable): {weights_stable}")
    print(f"Sum of weights: {weights_stable.sum()}")
    
    # Compute weighted gradient
    weighted_grad_stable = torch.zeros_like(grad_values[0])
    for i in range(n_samples):
        weighted_grad_stable += weights_stable[i] * grad_values[i]
    
    print(f"Weighted gradient (stable):\n{weighted_grad_stable}")
    
    print("\n" + "-" * 50)
    print("METHOD 3: USING PYTORCH'S LOGSUMEXP")
    print("-" * 50)
    
    # Using PyTorch's built-in logsumexp
    log_sum_exp = torch.logsumexp(log_p_values, dim=0)
    print(f"Logsumexp result: {log_sum_exp}")
    
    # Compute weights using logsumexp
    weights_logsumexp = torch.exp(log_p_values - log_sum_exp)
    print(f"Weights (logsumexp): {weights_logsumexp}")
    print(f"Sum of weights: {weights_logsumexp.sum()}")
    
    # Compute weighted gradient
    weighted_grad_logsumexp = torch.zeros_like(grad_values[0])
    for i in range(n_samples):
        weighted_grad_logsumexp += weights_logsumexp[i] * grad_values[i]
    
    print(f"Weighted gradient (logsumexp):\n{weighted_grad_logsumexp}")
    
    print("\n" + "-" * 50)
    print("COMPARISON")
    print("-" * 50)
    
    # Compare the methods
    print("Weight comparison:")
    print(f"Stable weights:      {weights_stable}")
    print(f"Logsumexp weights:   {weights_logsumexp}")
    
    if 'weights_unstable' in locals():
        print(f"Unstable weights:    {weights_unstable}")
    
    print("\nGradient comparison:")
    print(f"Stable vs Logsumexp gradient difference: {torch.norm(weighted_grad_stable - weighted_grad_logsumexp)}")
    
    if 'weighted_grad_unstable' in locals():
        print(f"Stable vs Unstable gradient difference: {torch.norm(weighted_grad_stable - weighted_grad_unstable)}")
    
    return {
        'weights_stable': weights_stable,
        'weights_logsumexp': weights_logsumexp,
        'grad_stable': weighted_grad_stable,
        'grad_logsumexp': weighted_grad_logsumexp
    }

def score_estimator_with_dummy_data():
    """
    Complete score estimator function using dummy data to demonstrate
    the stable computation method.
    """
    print("=" * 60)
    print("COMPLETE SCORE ESTIMATOR WITH DUMMY DATA")
    print("=" * 60)
    
    # Dummy parameters
    torch.manual_seed(42)
    n_samples = 64
    d = 3
    z = torch.randn(d, 2, 2) * 0.1
    theta = torch.randn(d, d) * 0.1
    data = {'x': torch.randn(100, d)}
    hparams = {
        'sigma_z': 1.0,
        'beta': 1.0,
        'alpha': 1.0,
        'n_mc_samples': n_samples
    }
    
    print(f"z shape: {z.shape}")
    print(f"theta shape: {theta.shape}")
    print(f"data['x'] shape: {data['x'].shape}")
    
    # Simulate the score estimator computation
    log_p_samples = []
    grad_z_samples = []
    
    for i in range(n_samples):
        # Dummy log joint probability (very small values)
        log_p = -1000 + torch.randn(1) * 2.0
        log_p_samples.append(log_p)
        
        # Dummy gradient of z
        grad_z = torch.randn_like(z) * 0.1
        grad_z_samples.append(grad_z)
    
    log_p_tensor = torch.stack(log_p_samples).squeeze()
    grad_z_tensor = torch.stack(grad_z_samples)
    
    print(f"\nLog probabilities range: [{log_p_tensor.min():.2f}, {log_p_tensor.max():.2f}]")
    print(f"Gradient tensor shape: {grad_z_tensor.shape}")
    
    # STABLE COMPUTATION
    print("\n" + "-" * 50)
    print("STABLE SCORE ESTIMATOR COMPUTATION")
    print("-" * 50)
    
    # 1. Compute stable weights using logsumexp
    log_p_max = torch.max(log_p_tensor)
    log_p_shifted = log_p_tensor - log_p_max
    p_shifted = torch.exp(log_p_shifted)
    weights = p_shifted / p_shifted.sum()
    
    print(f"Max log probability: {log_p_max:.2f}")
    print(f"Weights sum: {weights.sum():.6f}")
    print(f"Weight range: [{weights.min():.6f}, {weights.max():.6f}]")
    
    # 2. Compute weighted gradient
    weighted_grad = torch.zeros_like(grad_z_tensor[0])
    for i in range(n_samples):
        weighted_grad += weights[i] * grad_z_tensor[i]
    
    print(f"Final weighted gradient shape: {weighted_grad.shape}")
    print(f"Gradient norm: {torch.norm(weighted_grad):.6f}")
    
    return weighted_grad

# Run the demonstrations
if __name__ == "__main__":
    # Run the comparison
    results = dummy_score_estimator_stable_vs_unstable()
    
    print("\n" + "=" * 60)
    
    # Run the complete estimator
    final_grad = score_estimator_with_dummy_data()

SCORE ESTIMATOR: STABLE VS UNSTABLE COMPUTATION
Original log probabilities: tensor([-1000.5000, -1001.2000,  -999.8000, -1002.1000, -1000.9000])
Gradient values shape: torch.Size([5, 3, 2, 2])
Sample gradient values:
tensor([[[ 0.1927,  0.1487],
         [ 0.0901, -0.2106]],

        [[ 0.0678, -0.1235],
         [-0.0043, -0.1605]],

        [[-0.0752,  0.1649],
         [-0.0392, -0.1404]]])

--------------------------------------------------
METHOD 1: NUMERICALLY UNSTABLE
--------------------------------------------------
Direct exp(log_p): tensor([0., 0., 0., 0., 0.])
Sum of exp(log_p): 0.0
Weights (unstable): tensor([nan, nan, nan, nan, nan])
Sum of weights: nan
Weighted gradient (unstable):
tensor([[[nan, nan],
         [nan, nan]],

        [[nan, nan],
         [nan, nan]],

        [[nan, nan],
         [nan, nan]]])

--------------------------------------------------
METHOD 2: NUMERICALLY STABLE
--------------------------------------------------
Max log probability: -999.7999

In [42]:
import torch
import torch.nn.functional as F

def dummy_score_estimator_stable_vs_unstable():
    """
    Demonstrates numerically stable vs unstable score estimator computation
    with mathematically correct gradient averaging.
    """
    print("=" * 60)
    print("SCORE ESTIMATOR: STABLE VS UNSTABLE COMPUTATION")
    print("=" * 60)
    
    # Set random seed for reproducibility
    torch.manual_seed(42)
    
    # Dummy parameters
    n_samples = 5
    d = 3  # dimension of z
    z_dim = (d, 2, 2)  # shape of z tensor
    
    # Generate dummy log probabilities (very small values to demonstrate instability)
    log_p_values = torch.tensor([-1000.5, -1001.2, -999.8, -1002.1, -1000.9])
    print(f"Original log probabilities: {log_p_values}")
    
    # Generate dummy gradient values
    grad_values = torch.randn(n_samples, *z_dim) * 0.1
    print(f"Gradient values shape: {grad_values.shape}")
    print(f"Sample gradient values:\n{grad_values[0]}")
    
    print("\n" + "-" * 50)
    print("METHOD 1: NUMERICALLY UNSTABLE")
    print("-" * 50)
    
    # Unstable method: direct exponentiation
    try:
        p_unstable = torch.exp(log_p_values)
        print(f"Direct exp(log_p): {p_unstable}")
        print(f"Sum of exp(log_p): {p_unstable.sum()}")
        
        # Normalize to get weights
        weights_unstable = p_unstable / p_unstable.sum()
        print(f"Weights (unstable): {weights_unstable}")
        print(f"Sum of weights: {weights_unstable.sum()}")
        
        # Compute weighted gradient
        weighted_grad_unstable = torch.zeros_like(grad_values[0])
        for i in range(n_samples):
            weighted_grad_unstable += weights_unstable[i] * grad_values[i]
        
        print(f"Weighted gradient (unstable):\n{weighted_grad_unstable}")
        
    except Exception as e:
        print(f"Unstable method failed: {e}")
    
    print("\n" + "-" * 50)
    print("METHOD 2: NUMERICALLY STABLE")
    print("-" * 50)
    
    # Stable method: logsumexp trick
    log_p_max = torch.max(log_p_values)
    print(f"Max log probability: {log_p_max}")
    
    log_p_shifted = log_p_values - log_p_max
    print(f"Shifted log probabilities: {log_p_shifted}")
    
    p_shifted = torch.exp(log_p_shifted)
    print(f"Exp of shifted log_p: {p_shifted}")
    
    # Normalize to get weights
    weights_stable = p_shifted / p_shifted.sum()
    print(f"Weights (stable): {weights_stable}")
    print(f"Sum of weights: {weights_stable.sum()}")
    
    # Compute weighted gradient
    weighted_grad_stable = torch.zeros_like(grad_values[0])
    for i in range(n_samples):
        weighted_grad_stable += weights_stable[i] * grad_values[i]
    
    print(f"Weighted gradient (stable):\n{weighted_grad_stable}")
    
    print("\n" + "-" * 50)
    print("METHOD 3: USING PYTORCH'S LOGSUMEXP")
    print("-" * 50)
    
    # Using PyTorch's built-in logsumexp
    log_sum_exp = torch.logsumexp(log_p_values, dim=0)
    print(f"Logsumexp result: {log_sum_exp}")
    
    # Compute weights using logsumexp
    weights_logsumexp = torch.exp(log_p_values - log_sum_exp)
    print(f"Weights (logsumexp): {weights_logsumexp}")
    print(f"Sum of weights: {weights_logsumexp.sum()}")
    
    # Compute weighted gradient
    weighted_grad_logsumexp = torch.zeros_like(grad_values[0])
    for i in range(n_samples):
        weighted_grad_logsumexp += weights_logsumexp[i] * grad_values[i]
    
    print(f"Weighted gradient (logsumexp):\n{weighted_grad_logsumexp}")
    
    print("\n" + "-" * 50)
    print("COMPARISON")
    print("-" * 50)
    
    # Compare the methods
    print("Weight comparison:")
    print(f"Stable weights:      {weights_stable}")
    print(f"Logsumexp weights:   {weights_logsumexp}")
    
    if 'weights_unstable' in locals():
        print(f"Unstable weights:    {weights_unstable}")
    
    print("\nGradient comparison:")
    print(f"Stable vs Logsumexp gradient difference: {torch.norm(weighted_grad_stable - weighted_grad_logsumexp)}")
    
    if 'weighted_grad_unstable' in locals():
        print(f"Stable vs Unstable gradient difference: {torch.norm(weighted_grad_stable - weighted_grad_unstable)}")
    
    return {
        'weights_stable': weights_stable,
        'weights_logsumexp': weights_logsumexp,
        'grad_stable': weighted_grad_stable,
        'grad_logsumexp': weighted_grad_logsumexp
    }

def score_estimator_with_dummy_data():
    """
    Complete score estimator function using dummy data to demonstrate
    the stable computation method with correct gradient averaging.
    """
    print("=" * 60)
    print("COMPLETE SCORE ESTIMATOR WITH DUMMY DATA")
    print("=" * 60)
    
    # Dummy parameters
    torch.manual_seed(42)
    n_samples = 64
    d = 3
    z = torch.randn(d, 2, 2) * 0.1
    theta = torch.randn(d, d) * 0.1
    data = {'x': torch.randn(100, d)}
    hparams = {
        'sigma_z': 1.0,
        'beta': 1.0,
        'alpha': 1.0,
        'n_mc_samples': n_samples
    }
    
    print(f"z shape: {z.shape}")
    print(f"theta shape: {theta.shape}")
    print(f"data['x'] shape: {data['x'].shape}")
    
    # Simulate the score estimator computation
    log_p_samples = []
    grad_z_samples = []
    
    for i in range(n_samples):
        # Dummy log joint probability (very small values)
        log_p = -1000 + torch.randn(1) * 2.0
        log_p_samples.append(log_p)
        
        # Dummy gradient of z
        grad_z = torch.randn_like(z) * 0.1
        grad_z_samples.append(grad_z)
    
    log_p_tensor = torch.stack(log_p_samples).squeeze()
    grad_z_tensor = torch.stack(grad_z_samples)
    
    print(f"\nLog probabilities range: [{log_p_tensor.min():.2f}, {log_p_tensor.max():.2f}]")
    print(f"Gradient tensor shape: {grad_z_tensor.shape}")
    
    # STABLE COMPUTATION
    print("\n" + "-" * 50)
    print("STABLE SCORE ESTIMATOR COMPUTATION")
    print("-" * 50)
    
    # 1. Compute stable weights using logsumexp
    log_p_max = torch.max(log_p_tensor)
    log_p_shifted = log_p_tensor - log_p_max
    p_shifted = torch.exp(log_p_shifted)
    weights = p_shifted / p_shifted.sum()
    
    print(f"Max log probability: {log_p_max:.2f}")
    print(f"Weights sum: {weights.sum():.6f}")
    print(f"Weight range: [{weights.min():.6f}, {weights.max():.6f}]")
    
    # 2. Compute weighted gradient
    weighted_grad = torch.zeros_like(grad_z_tensor[0])
    for i in range(n_samples):
        weighted_grad += weights[i] * grad_z_tensor[i]
    
    print(f"Final weighted gradient shape: {weighted_grad.shape}")
    print(f"Gradient norm: {torch.norm(weighted_grad):.6f}")
    
    return weighted_grad

def verify_gradient_averaging():
    """
    Verify that the gradient averaging is mathematically correct
    by comparing with simple average and weighted average.
    """
    print("=" * 60)
    print("VERIFYING GRADIENT AVERAGING CORRECTNESS")
    print("=" * 60)
    
    torch.manual_seed(42)
    n_samples = 10
    d = 2
    z_dim = (d, 2)
    
    # Create simple test case with known probabilities
    log_p_values = torch.tensor([-1.0, -2.0, -1.5, -3.0, -1.2, -2.5, -1.8, -2.2, -1.3, -2.8])
    grad_values = torch.randn(n_samples, *z_dim)
    
    print(f"Log probabilities: {log_p_values}")
    print(f"Gradient shape: {grad_values.shape}")
    
    # Method 1: Simple average (equal weights)
    simple_avg = grad_values.mean(dim=0)
    print(f"\nSimple average gradient:\n{simple_avg}")
    
    # Method 2: Weighted average using stable method
    log_p_max = torch.max(log_p_values)
    log_p_shifted = log_p_values - log_p_max
    p_shifted = torch.exp(log_p_shifted)
    weights = p_shifted / p_shifted.sum()
    
    print(f"\nWeights: {weights}")
    print(f"Sum of weights: {weights.sum():.6f}")
    
    weighted_avg = torch.zeros_like(grad_values[0])
    for i in range(n_samples):
        weighted_avg += weights[i] * grad_values[i]
    
    print(f"Weighted average gradient:\n{weighted_avg}")
    
    # Method 3: Using PyTorch's softmax
    weights_softmax = F.softmax(log_p_values, dim=0)
    weighted_avg_softmax = torch.zeros_like(grad_values[0])
    for i in range(n_samples):
        weighted_avg_softmax += weights_softmax[i] * grad_values[i]
    
    print(f"\nSoftmax weights: {weights_softmax}")
    print(f"Weighted average (softmax):\n{weighted_avg_softmax}")
    
    # Verify consistency
    print(f"\nConsistency checks:")
    print(f"Weights vs Softmax weights difference: {torch.norm(weights - weights_softmax):.2e}")
    print(f"Weighted avg vs Softmax avg difference: {torch.norm(weighted_avg - weighted_avg_softmax):.2e}")
    
    # Show that weights sum to 1
    print(f"\nWeight sums:")
    print(f"Stable weights sum: {weights.sum():.6f}")
    print(f"Softmax weights sum: {weights_softmax.sum():.6f}")
    
    return {
        'simple_avg': simple_avg,
        'weighted_avg': weighted_avg,
        'weighted_avg_softmax': weighted_avg_softmax,
        'weights': weights,
        'weights_softmax': weights_softmax
    }

# Run the demonstrations
if __name__ == "__main__":
    # Run the comparison
    results = dummy_score_estimator_stable_vs_unstable()
    
    print("\n" + "=" * 60)
    
    # Run the complete estimator
    final_grad = score_estimator_with_dummy_data()
    
    print("\n" + "=" * 60)
    
    # Verify gradient averaging
    verification = verify_gradient_averaging()

SCORE ESTIMATOR: STABLE VS UNSTABLE COMPUTATION
Original log probabilities: tensor([-1000.5000, -1001.2000,  -999.8000, -1002.1000, -1000.9000])
Gradient values shape: torch.Size([5, 3, 2, 2])
Sample gradient values:
tensor([[[ 0.1927,  0.1487],
         [ 0.0901, -0.2106]],

        [[ 0.0678, -0.1235],
         [-0.0043, -0.1605]],

        [[-0.0752,  0.1649],
         [-0.0392, -0.1404]]])

--------------------------------------------------
METHOD 1: NUMERICALLY UNSTABLE
--------------------------------------------------
Direct exp(log_p): tensor([0., 0., 0., 0., 0.])
Sum of exp(log_p): 0.0
Weights (unstable): tensor([nan, nan, nan, nan, nan])
Sum of weights: nan
Weighted gradient (unstable):
tensor([[[nan, nan],
         [nan, nan]],

        [[nan, nan],
         [nan, nan]],

        [[nan, nan],
         [nan, nan]]])

--------------------------------------------------
METHOD 2: NUMERICALLY STABLE
--------------------------------------------------
Max log probability: -999.7999

## Let's assume the calculation for the score estimator is done as above
THe numerator and the denominator is needed to calculate 

In [ ]:
def grad_z_score_with_ratio_stable(z: torch.Tensor, data: Dict[str, Any], theta: torch.Tensor, hparams: Dict[str, Any]) -> torch.Tensor:
    """
    Computes ∇_z log p(Z,Θ|D) using equation (10) with stable ratio computation.
    
    Equation (10): ∇_Z log p(Z,Θ|D) = ∇_Z log p(Z) + [∇_Z E[p(Θ,D|G)]] / [E[p(Θ,D|G)]]
    """
    d = z.shape[0]
    n_samples = hparams.get('n_mc_samples', 64)

    # --- Gradient of the Prior ---
    grad_z_prior = -(z / hparams['sigma_z']**2)

    # --- Collect samples for both numerator and denominator ---
    log_joint_rewards = []  # log p(Θ,D|G) for each sample
    scores_list = []        # ∇_z log q(G|z) for each sample
    
    # Pre-compute probabilities to sample from
    with torch.no_grad():
        edge_probs = torch.sigmoid(scores(z, hparams))

    for _ in range(n_samples):
        # 1. Sample a hard graph G
        g_hard = torch.bernoulli(edge_probs)

        # 2. Calculate the score ∇_z log q(G|z)
        score = score_autograd_g_given_z(z, g_hard, hparams)

        # 3. Calculate log joint reward: log p(Θ,D|G)
        with torch.no_grad():
            log_joint_reward = (log_full_likelihood(data, g_hard, theta, hparams) + 
                               log_theta_prior(theta * g_hard, hparams.get('theta_prior_sigma')))
            
        log_joint_rewards.append(log_joint_reward)
        scores_list.append(score)

    # Convert to tensors
    log_joint_tensor = torch.stack(log_joint_rewards)  # (n_samples,)
    scores_tensor = torch.stack(scores_list)           # (n_samples, d, k, 2)
    
    # --- STABLE COMPUTATION OF THE RATIO ---
    
    # Step 1: Compute NUMERATOR = ∇_Z E_p(G|Z) [p(Θ,D|G)]
    # This is the score function estimator result
    # We need: Σ_i p(G_i|Z) * ∇_z log p(G_i|Z) * p(Θ,D|G_i)
    # Since we sampled from p(G|Z), this becomes: (1/M) * Σ_i ∇_z log p(G_i|Z) * p(Θ,D|G_i)
    
    # Convert log probabilities to actual probabilities using stable method
    log_max = torch.max(log_joint_tensor)
    log_shifted = log_joint_tensor - log_max
    p_shifted = torch.exp(log_shifted)  # These are p(Θ,D|G_i) / exp(log_max)
    
    # Numerator: weighted sum of scores by probabilities
    numerator = torch.zeros_like(scores_tensor[0])
    for i in range(n_samples):
        numerator += p_shifted[i] * scores_tensor[i]
    numerator = numerator / n_samples  # Average over samples
    
    # Step 2: Compute DENOMINATOR = E_p(G|Z) [p(Θ,D|G)]
    # This is: (1/M) * Σ_i p(Θ,D|G_i)
    denominator = p_shifted.sum() / n_samples
    
    # Step 3: Compute the ratio stably
    # We have: numerator/denominator, but both are scaled by exp(log_max)
    # So the ratio is: (numerator_scaled / exp(log_max)) / (denominator_scaled / exp(log_max))
    #                = numerator_scaled / denominator_scaled
    grad_likelihood_ratio = numerator / denominator
    
    # --- Final result ---
    total_grad = grad_z_prior + grad_likelihood_ratio
    
    return total_grad



In [44]:
# stable mean 




import torch

def stable_mean(fxs: torch.Tensor, dim: int = 0, keepdim: bool = False) -> torch.Tensor:
    """
    Numerically stable mean for arrays with mixed positive/negative values.
    
    Args:
        fxs: Input tensor
        dim: Dimension along which to compute the mean
        keepdim: Whether to keep the reduced dimension
    
    Returns:
        Stable mean of the input tensor
    """
    jitter = 1e-30
    
    def stable_mean_positive_only(fs: torch.Tensor, n: torch.Tensor) -> torch.Tensor:
        """Helper function for positive-only values"""
        # Add jitter to avoid log(0)
        fs_safe = fs + jitter
        log_sum = torch.logsumexp(torch.log(fs_safe), dim=dim, keepdim=True)
        log_n = torch.log(n + jitter)
        return torch.exp(log_sum - log_n)
    
    # Separate positive and negative values
    positive_mask = fxs > 0.0
    negative_mask = fxs < 0.0
    
    f_xs_positive = fxs * positive_mask.float()
    f_xs_negative = -fxs * negative_mask.float()  # Make negative values positive
    
    # Count positive and negative elements
    n_positive = positive_mask.sum(dim=dim, keepdim=True).float()
    n_negative = negative_mask.sum(dim=dim, keepdim=True).float()
    
    # Total number of elements
    if dim is None:
        n_total = torch.tensor(fxs.numel(), dtype=fxs.dtype, device=fxs.device)
    else:
        n_total = torch.tensor(fxs.shape[dim], dtype=fxs.dtype, device=fxs.device)
    
    # Compute stable means for positive and negative parts
    avg_positive = stable_mean_positive_only(f_xs_positive, n_positive)
    avg_negative = stable_mean_positive_only(f_xs_negative, n_negative)
    
    # Combine with proper weighting
    result = (n_positive / n_total) * avg_positive - (n_negative / n_total) * avg_negative
    
    return result if keepdim else result.squeeze(dim)


def log_stable_mean_from_logs(fsx: torch.Tensor, dim: int = 0) -> torch.Tensor:
    """
    Compute log(mean(exp(fsx))) stably when fsx contains log values.
    
    This is equivalent to computing the log of the mean of probabilities
    when fsx contains log probabilities.
    
    Args:
        fsx: Input tensor containing log values
        dim: Dimension along which to compute the mean
    
    Returns:
        Log of the mean of exp(fsx)
    """
    
    n = fsx.shape[dim]
    
    # Compute logsumexp and subtract log(n) for the mean
    lse = torch.logsumexp(fsx, dim=dim)
    log_n = torch.log(torch.tensor(n, dtype=fsx.dtype, device=fsx.device))
    
    return lse - log_n


def stable_mean_simple(x: torch.Tensor, dim: int = 0, keepdim: bool = False) -> torch.Tensor:
    """
    Simplified stable mean that works well for most cases.
    Similar to the one in your models/utils.py but cleaner.
    """
    jitter = 1e-30
    
    if not x.is_floating_point():
        x = x.float()
    
    # Separate positive and negative parts
    pos = x.clamp(min=0) + jitter
    neg = (-x).clamp(min=0) + jitter
    
    # Compute stable sums using logsumexp
    sum_pos = torch.exp(torch.logsumexp(torch.log(pos), dim=dim, keepdim=True))
    sum_neg = torch.exp(torch.logsumexp(torch.log(neg), dim=dim, keepdim=True))
    
    # Get the count
    n = torch.tensor(x.shape[dim] if dim < x.dim() else x.numel(), 
                     dtype=x.dtype, device=x.device)
    
    # Compute mean
    mean = (sum_pos - sum_neg) / (n + jitter)
    
    return mean if keepdim else mean.squeeze(dim)





In [47]:
## try the stable mean function 

dummy_tensor = torch.tensor([-1000, -1000, -1000, -1000, -1000])

print(f'dummy_tensor: {dummy_tensor}')

print(f'stable_mean: {stable_mean(dummy_tensor)}')

print(f'stable_mean')



dummy_tensor: tensor([-1000, -1000, -1000, -1000, -1000])
stable_mean: -999.9996337890625
stable_mean exp: 0.0


In [52]:
import torch

def logsumexp_with_sign(a, b, dim=-1, keepdim=False, eps=1e-40):
    """
    Stable log|Σ b·exp(a)|   and   sign(Σ b·exp(a))
    ------------------------------------------------
    a   : tensor – the log-terms (ℓ_j)
    b   : tensor – the weights   (b_j)  (can be ±)
          `a` and `b` must be broadcast–compatible.
    dim : dimension over which to sum
    """
    a_max = torch.max(a, dim=dim, keepdim=True)[0]           # shift
    scaled = b * torch.exp(a - a_max)                        # weights in safe range
    s      = scaled.sum(dim=dim, keepdim=keepdim)            # may be ±
    sign   = torch.sign(s).detach()                          # ±1 (detach ⇢ no grad through sign)
    logabs = torch.log(torch.abs(s) + eps) + a_max.squeeze(dim) \
             if not keepdim else torch.log(torch.abs(s) + eps) + a_max
    return logabs, sign


In [55]:
# --- dummy data for demonstration ---
K          = 5                          # number of graph samples
flat_dim   = 12                         # d*k*2  after you reshape

torch.manual_seed(0)
grad_z     = torch.randn(flat_dim, K)   # [flat_dim, K]  each row = score_row
log_lik    = torch.randn(K) * -2.0      # [K]

# --- denominator: plain logsumexp because weights are +1 ---
log_den    = torch.logsumexp(log_lik, dim=0)          # scalar  log D

# --- numerator: row-wise signed logsumexp ---
log_num, sign_num = [], []
for r in range(flat_dim):
    ln, sg = logsumexp_with_sign(grad_z[r], log_lik)
    log_num.append(ln)
    sign_num.append(sg)

log_num = torch.stack(log_num)    # [flat_dim]
sign_num= torch.stack(sign_num)   # [flat_dim]  (+1/-1/0)
print(f'log_num: {log_num}')
print(f'sign_num: {sign_num}')
print(f'grad_z: {grad_z}')

# --- weighted average  N/D  (still on-graph wrt log_lik and grad_z) ---
weighted_score_flat = sign_num * torch.exp(log_num - log_den)   # [flat_dim]

# reshape back to [d, k, 2] if you like
d, k = 3, 2
weighted_score = weighted_score_flat.view(d, k, 2)

print(weighted_score.shape)   # torch.Size([3, 2, 2])
print(weighted_score)


log_num: tensor([-2.0601, -0.6257, -2.0684, -0.0838,  1.4336, -0.1034,  0.1488,  0.1566,
         1.1128, -1.3828,  1.5209,  0.6923])
sign_num: tensor([ 1., -1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1., -1.])
grad_z: tensor([[-1.1258, -1.1524, -0.2506, -0.4339,  0.8487],
        [ 0.6920, -0.3160, -2.1152,  0.3223, -1.2633],
        [ 0.3500,  0.3081,  0.1198,  1.2377,  1.1168],
        [-0.2473, -1.3527, -1.6959,  0.5667,  0.7935],
        [ 0.5988, -1.5551, -0.3414,  1.8530,  0.7502],
        [-0.5855, -0.1734,  0.1835,  1.3894,  1.5863],
        [ 0.9463, -0.8437, -0.6136,  0.0316, -0.4927],
        [ 0.2484,  0.4397,  0.1124,  0.6408,  0.4412],
        [-0.1023,  0.7924, -0.2897,  0.0525,  0.5943],
        [ 1.5419,  0.5073, -0.5910, -0.5692,  0.9200],
        [ 1.1108,  1.2899, -1.4959, -0.1938,  0.4455],
        [ 1.3253, -1.6293, -0.5497, -0.4798, -0.4997]])
torch.Size([3, 2, 2])
tensor([[[ 0.0134, -0.0562],
         [-0.0133, -0.0967]],

        [[-0.4408, -0.0948],
       

In [ ]:
import numpy as np 








def stable_mean(fxs):
    # assumes fs are only positive
    jitter = 1e-30

    # Taking n separately we need non-zero
    stable_mean_psve_only = lambda fs, n: np.exp(
        np.logsumexp(np.log(fs)) - np.log(n + jitter)
    )

    f_xs_psve = fxs * (fxs > 0.0)
    f_xs_ngve = -fxs * (fxs < 0.0)

    n_psve = np.sum((fxs > 0.0))
    n_ngve = fxs.size - n_psve

    avg_psve = stable_mean_psve_only(f_xs_psve, n_psve)
    avg_ngve = stable_mean_psve_only(f_xs_ngve, n_ngve)
    return (n_psve / fxs.size) * avg_psve - (n_ngve / fxs.size) * avg_ngve


def log_stable_mean_from_logs(fsx):
    lse = np.logsumexp(fsx)
    return lse - fsx.size

In [129]:

def stable_ratio(grad_samples, log_density_samples):
    eps = 1e-30
    
    log_p = torch.stack(log_density_samples)
    grads = torch.stack(grad_samples)

    log_grads_abs = torch.log(grads.abs() + eps)

    log_grads_abs += log_p

    pos_mask = grads >= 0
    neg_mask = grads < 0

    n_pos = pos_mask.sum().clamp(min=1)
    n_neg = neg_mask.sum().clamp(min=1)

    print(f'n_pos: {n_pos}, n_neg: {n_neg}')

    print(f'log_grads_abs: {log_grads_abs}')

    log_grads_abs_pos = log_grads_abs.masked_fill(~pos_mask, float('-inf'))
    print(f'log_grads_abs_pos: {log_grads_abs_pos}')
    log_grads_abs_neg = log_grads_abs.masked_fill(~neg_mask, float('-inf'))
    print(f'log_grads_abs_neg: {log_grads_abs_neg}')

    log_numerator_positive = torch.logsumexp(log_grads_abs_pos, dim=0) - torch.log(n_pos.float())
    print(f'log_numerator_positive: {log_numerator_positive}')
    log_numerator_negative = torch.logsumexp(log_grads_abs_neg, dim=0) - torch.log(n_neg.float())
    print(f'log_numerator_negative: {log_numerator_negative}')
    log_den = torch.logsumexp(log_p, dim=0) - torch.log(torch.tensor(len(log_p), dtype=log_p.dtype, device=log_p.device))
    print(f'log_den: {log_den}')


    # can we use the log_p - log_p_max for the denominator
    log_p_max = log_p.max()
    log_p_shifted = log_p - log_p_max
    log_den_shifted = torch.logsumexp(log_p_shifted, dim=0) - torch.log(torch.tensor(len(log_p), dtype=log_p.dtype, device=log_p.device))
    log_den_shifted = log_den_shifted + log_p_max
    print(f'log_den_shifted: {log_den_shifted}')

    log_den_shifted_exp = torch.exp(log_den_shifted)
    print(f'log_den_shifted_exp: {log_den_shifted_exp}')



    final_grad = torch.exp(log_numerator_positive - log_den) - torch.exp(log_numerator_negative - log_den)
    print(f'final_grad: {final_grad}')

    final_grad_shifted = torch.exp(log_numerator_positive - log_den_shifted) - torch.exp(log_numerator_negative - log_den_shifted)
    print(f'final_grad_shifted: {final_grad_shifted}')

    #print(f'n_pos: {n_pos}, n_neg: {n_neg}')





  
    #print(f"grad_samples: {grad_samples}")
    #print(f'grads: {grads}')
    #print(f'pos_mask: {pos_mask}')
    #print(f'pos_mask logs: {pos_mask.log()}')


    #print(f'neg_mask: {neg_mask}')
    #print(f'neg_mask logs: {neg_mask.log()}')



    #print(f'log_grads_abs: {log_grads_abs}')


    #print(f'end of grads')x
    #print(f'--------------------------------')


    #print(f'log_p: {log_p}')

    #print(f'grads: {grads}')

    return final_grad


torch.manual_seed(0)

num_samples = 50

# Create grad samples with net positive mean
grad_samples = [x.clone().detach() for x in torch.linspace(-20, 10, num_samples)]  # More positive values than negative

# Create log density samples from N(-1000, 100)
log_density_samples = [torch.normal(mean=-1000.0, std=10.0, size=()) for _ in range(num_samples)]

result = stable_ratio(grad_samples, log_density_samples)



n_pos: 17, n_neg: 33
log_grads_abs: tensor([ 2.9957,  2.9646,  2.9326,  2.8994,  2.8651,  2.8296,  2.7928,  2.7546,
         2.7148,  2.6734,  2.6303,  2.5852,  2.5379,  2.4883,  2.4361,  2.3811,
         2.3228,  2.2609,  2.1950,  2.1243,  2.0484,  1.9661,  1.8765,  1.7781,
         1.6689,  1.5463,  1.4065,  1.2440,  1.0498,  0.8087,  0.4902,  0.0202,
        -0.8961, -1.5892, -0.2029,  0.3567,  0.7133,  0.9757,  1.1834,  1.3552,
         1.5018,  1.6296,  1.7430,  1.8448,  1.9371,  2.0217,  2.0996,  2.1720,
         2.2394,  2.3026])
log_grads_abs_pos: tensor([   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,
           -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,
           -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,
           -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,
           -inf, -1.5892, -0.2029,  0.3567,  0.7133,  0.9757,  1.1834,  1.3552,
         1.5018,  1.6296,  1.7430,  1.

In [ ]:

def stable_ratio(grad_samples, log_density_samples):
    eps = 1e-30
    
    log_p = torch.stack(log_density_samples)
    grads = torch.stack(grad_samples)

    log_grads_abs = torch.log(grads.abs() + eps)

    pos_mask = grads >= 0
    neg_mask = grads < 0

    n_pos = pos_mask.sum().clamp(min=1)
    n_neg = neg_mask.sum().clamp(min=1)

    print(f'n_pos: {n_pos}, n_neg: {n_neg}')

    print(f'log_grads_abs: {log_grads_abs}')

    log_grads_abs_pos = log_grads_abs.masked_fill(~pos_mask, float('-inf'))
    print(f'log_grads_abs_pos: {log_grads_abs_pos}')
    log_grads_abs_neg = log_grads_abs.masked_fill(~neg_mask, float('-inf'))
    print(f'log_grads_abs_neg: {log_grads_abs_neg}')

    log_numerator_positive = torch.logsumexp(log_grads_abs_pos, dim=0) - torch.log(n_pos.float())
    print(f'log_numerator_positive: {log_numerator_positive}')
    log_numerator_negative = torch.logsumexp(log_grads_abs_neg, dim=0) - torch.log(n_neg.float())
    print(f'log_numerator_negative: {log_numerator_negative}')
    log_den = torch.logsumexp(log_p, dim=0) - torch.log(torch.tensor(len(log_p), dtype=log_p.dtype, device=log_p.device))
    print(f'log_den: {log_den}')





    final_grad = torch.exp(log_numerator_positive - log_den) - torch.exp(log_numerator_negative - log_den)
    print(f'final_grad: {final_grad}')


    return final_grad



In [ ]:
import torch

def stable_ratio_fixed(grad_samples, log_density_samples):
    eps = 1e-30
    
    log_p = torch.stack(log_density_samples)
    grads = torch.stack(grad_samples)

    log_grads_abs = torch.log(grads.abs() + eps)

    pos_mask = grads >= 0
    neg_mask = grads < 0

    n_pos = pos_mask.sum().clamp(min=1)
    n_neg = neg_mask.sum().clamp(min=1)

    log_grads_abs_pos = log_grads_abs.masked_fill(~pos_mask, float('-inf'))
    log_grads_abs_neg = log_grads_abs.masked_fill(~neg_mask, float('-inf'))

    # Numerator terms in log space
    log_num_pos = torch.logsumexp(log_grads_abs_pos, dim=0) - torch.log(n_pos.float())
    log_num_neg = torch.logsumexp(log_grads_abs_neg, dim=0) - torch.log(n_neg.float())
    
    # Denominator in log space
    log_den = torch.logsumexp(log_p, dim=0) - torch.log(torch.tensor(len(log_p), dtype=log_p.dtype, device=log_p.device))

    # Perform subtraction in log-space to avoid overflow/underflow
    # This computes log(num_pos - num_neg)
    # We must handle the case where the result is negative or zero
    if log_num_pos > log_num_neg:
        # Using log(exp(a) - exp(b)) = a + log(1 - exp(b-a))
        log_numerator = log_num_pos + torch.log1p(-torch.exp(log_num_neg - log_num_pos))
        sign = 1.0
    else:
        # To handle num_neg > num_pos
        log_numerator = log_num_neg + torch.log1p(-torch.exp(log_num_pos - log_num_neg))
        sign = -1.0

    # Final calculation: log(numerator) - log(denominator)
    final_log_grad = log_numerator - log_den
    
    return sign * torch.exp(final_log_grad)

# Using your test case
torch.manual_seed(0)
num_samples = 50
grad_samples = [torch.tensor(x) for x in torch.linspace(-4, 10, num_samples)]
## change first 20 to more *2


log_density_samples = [torch.normal(mean=-1000.0, std=100.0, size=()) for _ in range(num_samples)]

result = stable_ratio_fixed(grad_samples, log_density_samples)
print(f'Corrected Final Gradient: {result}')

TypeError: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

In [120]:
import math
def stable_mean(x: torch.Tensor, eps: float = 1e-30) -> torch.Tensor:
    """
    Numerically-stable mean of `x` (any shape, any sign).
    Works by treating positive and negative parts separately and
    using log-sum-exp for each part.
    Returns a scalar.
    """
    x = x.flatten()

    pos_mask = x >= 0
    neg_mask = ~pos_mask                # same as x < 0

    pos_vals = x[pos_mask]              # ≥ 0
    neg_vals = -x[neg_mask]             # magnitudes of the negatives

    n_pos = pos_vals.numel()
    n_neg = neg_vals.numel()
    n_tot = n_pos + n_neg
    if n_tot == 0:
        raise ValueError("`x` is empty!")

    # ----  log-mean of the positive part  ---------------------------------
    if n_pos:
        log_pos_mean = (
            torch.logsumexp(torch.log(pos_vals + eps), dim=0) -
            math.log(n_pos)
        )
        pos_mean = log_pos_mean.exp()
    else:
        pos_mean = torch.tensor(0., dtype=x.dtype, device=x.device)

    # ----  log-mean of the |negative| part  -------------------------------
    if n_neg:
        log_neg_mean = (
            torch.logsumexp(torch.log(neg_vals + eps), dim=0) -
            math.log(n_neg)
        )
        neg_mean = log_neg_mean.exp()
    else:
        neg_mean = torch.tensor(0., dtype=x.dtype, device=x.device)

    # unconditional mean =  P(+)·E[+ ]  –  P(−)·E[|−|]
    return (n_pos / n_tot) * pos_mean - (n_neg / n_tot) * neg_mean


def log_mean_from_logs(log_x, eps: float = 1e-30):
    """log E[x] where `log_x = log(x)` and x≥0."""
    return torch.logsumexp(log_x, dim=0) - math.log(log_x.shape[0])

def stable_ratio_with_stable_mean(grad_samples, log_density_samples):
    grads = torch.stack(grad_samples).flatten()

    # numerator  – ordinary stable mean (takes ± values)
    num = stable_mean(grads)

    # denominator – stay in log-space until the very end
    log_p = torch.stack(log_density_samples).flatten()
    log_p_max = log_p.max()
    log_p = log_p - log_p_max
    log_den = log_mean_from_logs(log_p)     # log E[p]
    den = torch.exp(log_den)               # E[p] in ℝ

    print(f'num: {num}, den: {den}')
    print(f'den: {den} max p exp: {torch.exp(log_p_max)}')

    return num / den

# 50 log-density samples ~ N(-1000, 100)
log_density_samples = [torch.normal(mean=-1000.0, std=100.0, size=(50,))]

# 50 gradient samples (some positive, some negative)
grad_samples = [torch.linspace(-4, 10, steps=50)]  # evenly spaced, centered at 0

# Print and test

# Print and test
print(f'log_density_samples: {log_density_samples[0][:5]} ...')
print(f'grad_samples: {grad_samples[0][:5]} ...')

result = stable_ratio_with_stable_mean(grad_samples, log_density_samples)
print(f'\nFinal Result: {result}')







log_density_samples: tensor([ -923.8889,  -938.1699, -1029.9385, -1018.7778,  -808.4102]) ...
grad_samples: tensor([-4.0000, -3.7143, -3.4286, -3.1429, -2.8571]) ...
num: 3.000000238418579, den: 0.019999999552965164
den: 0.019999999552965164 max p exp: 0.0

Final Result: 150.00001525878906


In [83]:
import torch
import math

def test_stable_ratio_functions():
    """
    Quick test to verify both stable ratio functions work correctly
    """
    print("=" * 60)
    print("TESTING STABLE RATIO FUNCTIONS")
    print("=" * 60)
    
    # Test case 1: Mixed positive/negative gradients with very small log densities
    print("\nTest Case 1: Mixed gradients with small log densities")
    print("-" * 50)
    
    grad_samples = [
        torch.tensor([15.5, -8.2, 3.1, -12.7, 0.5]),
        torch.tensor([-2.3, 9.8, -15.1, 4.6, -0.8]),
        torch.tensor([7.2, -5.4, 11.3, -1.9, 6.0])
    ]
    
    log_density_samples = [
        torch.tensor([-1001.5, -999.8, -1000.2, -1002.1, -1000.9]),
        torch.tensor([-1000.3, -998.7, -1001.8, -999.5, -1000.6]),
        torch.tensor([-999.9, -1001.2, -998.3, -1000.7, -999.4])
    ]
    
    print(f"Gradient shapes: {[g.shape for g in grad_samples]}")
    print(f"Log density ranges: {[f'[{lg.min():.1f}, {lg.max():.1f}]' for lg in log_density_samples]}")
    
    # Test both functions
    try:
        result1 = stable_ratio(grad_samples, log_density_samples)
        print(f"stable_ratio result: {result1}")
        print(f"stable_ratio result shape: {result1.shape}")
    except Exception as e:
        print(f"stable_ratio failed: {e}")
        result1 = None
    
    try:
        result2 = stable_ratio_with_stable_mean(grad_samples, log_density_samples)
        print(f"stable_ratio_with_stable_mean result: {result2}")
        print(f"stable_ratio_with_stable_mean result shape: {result2.shape}")
    except Exception as e:
        print(f"stable_ratio_with_stable_mean failed: {e}")
        result2 = None
    
    if result1 is not None and result2 is not None:
        diff = torch.norm(result1 - result2)
        print(f"Difference between methods: {diff}")
        print(f"Results are close: {diff < 1e-5}")
    
    # Test case 2: Naive comparison (will likely fail due to numerical issues)
    print("\nTest Case 2: Comparison with naive method")
    print("-" * 50)
    
    try:
        # Naive unstable computation
        grads_flat = torch.cat([g.flatten() for g in grad_samples])
        log_densities_flat = torch.cat([ld.flatten() for ld in log_density_samples])
        densities_flat = torch.exp(log_densities_flat)
        
        naive_num = grads_flat.mean()
        naive_den = densities_flat.mean()
        naive_result = naive_num / naive_den if naive_den != 0 else torch.tensor(float('inf'))
        
        print(f"Naive numerator: {naive_num}")
        print(f"Naive denominator: {naive_den}")
        print(f"Naive result: {naive_result}")
        
        if result1 is not None:
            print(f"Stable vs Naive difference: {torch.norm(result1.flatten().mean() - naive_result)}")
        
    except Exception as e:
        print(f"Naive method failed (expected): {e}")
    
    # Test case 3: Simple case for validation
    print("\nTest Case 3: Simple validation case")
    print("-" * 50)
    
    simple_grads = [torch.tensor([1.0, -1.0, 2.0])]
    simple_log_densities = [torch.tensor([-1.0, -2.0, -1.5])]  # Less extreme values
    
    try:
        simple_result1 = stable_ratio(simple_grads, simple_log_densities)
        simple_result2 = stable_ratio_with_stable_mean(simple_grads, simple_log_densities)
        
        print(f"Simple case - stable_ratio: {simple_result1}")
        print(f"Simple case - stable_mean version: {simple_result2}")
        print(f"Simple case difference: {torch.norm(simple_result1 - simple_result2)}")
        
        # Manual calculation for verification
        grads = torch.tensor([1.0, -1.0, 2.0])
        log_p = torch.tensor([-1.0, -2.0, -1.5])
        p = torch.exp(log_p)
        
        manual_num = grads.mean()
        manual_den = p.mean()
        manual_result = manual_num / manual_den
        
        print(f"Manual calculation: {manual_result}")
        
    except Exception as e:
        print(f"Simple case failed: {e}")

# Run the test
test_stable_ratio_functions()

TESTING STABLE RATIO FUNCTIONS

Test Case 1: Mixed gradients with small log densities
--------------------------------------------------
Gradient shapes: [torch.Size([5]), torch.Size([5]), torch.Size([5])]
Log density ranges: ['[-1002.1, -999.8]', '[-1001.8, -998.7]', '[-1001.2, -998.3]']
stable_ratio result: tensor([nan, nan, nan, nan, nan])
stable_ratio result shape: torch.Size([5])
stable_ratio_with_stable_mean result: inf
stable_ratio_with_stable_mean result shape: torch.Size([])
Difference between methods: nan
Results are close: False

Test Case 2: Comparison with naive method
--------------------------------------------------
Naive numerator: 0.7733333706855774
Naive denominator: 0.0
Naive result: inf
Stable vs Naive difference: nan

Test Case 3: Simple validation case
--------------------------------------------------
Simple case - stable_ratio: tensor([ 1.3591, -7.3891,  4.4817])
Simple case - stable_mean version: 2.7535126209259033
Simple case difference: 10.382801055908203
Ma